# Bird Box
Helping drone detects bird in the field of view, and potentially draw a plan to avoid it in advance.

In [ ]:
# data source: NABirds data set provided by the Cornell Lab of Ornithology
# https://dl.allaboutbirds.org/nabirds
# https://www.birds.cornell.edu/home
